## IMPORTS:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from sklearn.linear_model import LogisticRegression,SGDClassifier,Perceptron,PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import LinearSVC, NuSVC
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,ExtraTreesClassifier
from sklearn.neighbors import RadiusNeighborsClassifier,NearestCentroid
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import accuracy_score,confusion_matrix, plot_confusion_matrix
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from statistics import mode
import pandas as pd
import seaborn as sns

## Models:

In [4]:
def pickling (pickle_name,obj):
  save_obj=open('/content/drive/MyDrive/Fake_News/pickles/'+pickle_name+".pickle","wb")
  pickle.dump(obj,save_obj)
  save_obj.close()
def depickling(pickle_name):
  f=open('/content/drive/MyDrive/Fake_News/pickles/'+pickle_name+'.pickle',"rb")
  obj=pickle.load(f)
  f.close()
  return obj

In [5]:
acc_dict=depickling("acc_dict")
time_dict=depickling("time_dict")

In [6]:
vectorize = TfidfVectorizer()
xv_train = vectorize.fit_transform(depickling("xtrain"))
xv_test = vectorize.transform(depickling("xtest"))
ytrain=depickling("ytrain")
ytest=depickling("ytest")

In [7]:
import time

def train_and_eval(classifier,name):
  print(name)
  print()
  start=time.time()
  classifier.fit(xv_train,ytrain)
  stop=time.time()
  time_dict[name]=stop-start
  ypred=classifier.predict(xv_test)
  acc_dict[name]=100*accuracy_score(ypred,ytest)
  pickling(name,classifier)
  print("Accuracy of ",name," model = ", acc_dict[name], "%")
  print("Time of ",name," model = ", time_dict[name], "sec")
  #classifier.get_params()
  #plt.figure(figsize = (4, 4))
  #filterwarnings('ignore')
  #plot_confusion_matrix(classifier, xv_test, ytest, cmap = 'Blues')
  print()

In [ ]:
classifiers=[LogisticRegression(),MultinomialNB(),BernoulliNB(),SGDClassifier(),LinearSVC(),NuSVC(),
  RandomForestClassifier(),AdaBoostClassifier(),Perceptron(),PassiveAggressiveClassifier(),
  NearestCentroid(),BaggingClassifier(),ExtraTreesClassifier()]
names=["LogiReg","MNB","BNB","SGDC","LinSVC","nuSVC","forest","ada","Perceptron","PassiveAggressive",
       "NearCent","Bagging","ExtraTree"]

In [ ]:
for i in range (0,len(names)):
  train_and_eval(classifiers[i],names[i])
  pickling(names[i],classifiers[i])
pickling("acc_dict",acc_dict)
pickling("time_dict",time_dict)

In [ ]:
class VoteClassifier ():
  def __init__(self,arr_model):
    self._classifiers=arr_model
  def classify(self,xtest):
    ans=[]
    i=0
    for x in xtest:
      votes=[]
      for c in self._classifiers:
       v=c.predict(x.reshape(1,-1))
       votes.append(v[0])
      ans.append(mode(votes))
      print(i)
      i+=1
    return ans

In [ ]:
models=[]
for name in names:
  models.append(depickling(name)) 
comb=VoteClassifier(models)
res=comb.classify(xv_test)

KeyboardInterrupt: ignored

In [ ]:
pickling("comb_classifier_ypred",res)
ytest=list(ytest)

In [ ]:
match=0
for i in range (0,len(res)):
  if res[i]==ytest[i]:
    match+=1
comb_acc=match/len(res)*100
acc_dict["comb"]=comb_acc
time_dict["comb"]=960
print(comb_acc)

98.57697283311772


## Comparison

In [ ]:
acc=[]
time=[]
acc_by_time=[]
for model in acc_dict.keys():
  acc.append(acc_dict[model])
  time.append(time_dict[model])

In [ ]:
plt.figure(figsize=(20,6))
df={}
df["Algorithms"]=list(acc_dict.keys())
df["Accuracy"]=acc
df["time"]=time

df=pd.DataFrame(df,columns=['Algorithms','Accuracy','time','ratio'])
df["ratio"]=df["Accuracy"]*10/df["time"]
def ploty(Y):
  plt.figure(figsize=(20,6))
  plot=sns.barplot(x="Algorithms",y=Y,data=df)
  for i in plot.patches:
    plot.annotate(round(i.get_height(),5),(i.get_x()+0.5*i.get_width(),i.get_height()*2/3),
                ha='center', va='center')
  plt.show()

ploty("Accuracy")
ploty("time")
ploty("ratio")